## Algorithmic Trading: Mean Reversion 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [ ]:
START_TIME = '2024-05-01'
END_TIME = '2024-08-15'
HTF_MA = 100
HTF_MA_TYPE = 'close'
HTF_MA_WINDOW = 10
RSI_PERIOD = 13
RSI_SELL_LEVEL = 70
RSI_BUY_LEVEL = 30
BUY_TREND_ANGLE = 50
SELL_TREND_ANGLE = -50
maxTrades = 5
initial_balance = 100000 

### Import Libraries

### Get Data

In [ ]:
# Load data from CSV
csv_file_path = 'USA30IDXUSD.csv'
csv_file_path4h = 'USA30IDXUSD4h.csv'
csv_file_tickData = 'processed_tick_data.csv'
df_ltf = pd.read_csv(csv_file_path)
df_htf = pd.read_csv(csv_file_path4h)
#df_tickdata = dd.read_csv(csv_file_tickData)

Format HTF Data

In [ ]:
# Combine Date and Timestamp columns into a single datetime column
df_htf['date'] = pd.to_datetime(df_htf['Date'].astype(str) + ' ' + df_htf['Timestamp'].astype(str))

# Drop the original Date and Timestamp columns if no longer needed
df_htf.drop(['Date', 'Timestamp'], axis=1, inplace=True)

# Rename columns
df_htf = df_htf.rename(columns={
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume'
})

# Reverse index
df_htf = df_htf.reindex(index=df_htf.index[::-1])
df_htf.reset_index(level=0, inplace=True)

# Drop the 'index' column if it's not needed
df_htf.drop('index', axis=1, inplace=True)


format LTF Data

In [ ]:
df_ltf['date'] = pd.to_datetime(df_ltf['Date'].astype(str) + ' ' + df_ltf['Timestamp'].astype(str))

df_ltf.drop(['Date', 'Timestamp'], axis=1, inplace=True)

df_ltf = df_ltf.rename(columns={
    'Open': 'open',
    'High': 'high',
    'Low': 'low',
    'Close': 'close',
    'Volume': 'volume'
})

df_ltf = df_ltf.reindex(index=df_ltf.index[::-1])
df_ltf.reset_index(level=0, inplace=True)

df_ltf.drop('index', axis=1, inplace=True)

Select Start and End Time

In [ ]:
df_ltf = df_ltf[((df_ltf['date'] >= pd.to_datetime(START_TIME)) & (df_ltf['date'] <= pd.to_datetime(END_TIME)))]
df_htf = df_htf[((df_htf['date'] >= pd.to_datetime(START_TIME)) & (df_htf['date'] <= pd.to_datetime(END_TIME)))]

### Calculate ma

In [ ]:
df_htf[f'EMA_{HTF_MA}'] = df_htf[HTF_MA_TYPE].ewm(span=HTF_MA, adjust=False).mean()

Calculate the Angle of the Linear regression 

In [ ]:
def calculate_ma_angle(df, ma_column=f'EMA_{HTF_MA}', window=10):
    angles = np.full(len(df), np.nan)

    for i in range(len(df)):
        if i + window < len(df):  
            y_values = [df[ma_column].iloc[i], df[ma_column].iloc[i + window]]
            X = np.array([0, window]).reshape(-1, 1) 
            model = LinearRegression().fit(X, y_values)
            slope = model.coef_[0] 
            angle_rad = np.arctan(slope)
            angle_deg = np.degrees(angle_rad)
            angle_deg = angle_deg*(-1)
            angles[i] = angle_deg

        else:
            angles[i] = np.nan
    
    return angles

df_htf['ma_angle'] = calculate_ma_angle(df_htf, ma_column=f'EMA_{HTF_MA}')

In [ ]:
plt.figure(figsize=(12, 5))
plt.xticks(rotation=45)
plt.plot(df_htf['date'], df_htf['close'], label='Close')
plt.plot(df_htf['date'], df_htf[f'EMA_{HTF_MA}'], label=f'{HTF_MA}-period EMA')

for i in range(len(df_htf) - 1):
    angle = df_htf['ma_angle'].iloc[i]
    if angle > 50:
        plt.axvspan(df_htf['date'].iloc[i], df_htf['date'].iloc[i + 1], color='lightgreen', alpha=0.5)  # Bullish: Light Green
    elif angle < -50:
        plt.axvspan(df_htf['date'].iloc[i], df_htf['date'].iloc[i + 1], color='lightcoral', alpha=0.5)  # Bearish: Light Coral
    else:
        plt.axvspan(df_htf['date'].iloc[i], df_htf['date'].iloc[i + 1], color='darkgrey', alpha=0.5)  # Ranging: Dark Grey

for i in range(0, len(df_htf), max(1, len(df_htf) // 10)):
    plt.annotate(f'{df_htf["ma_angle"].iloc[i]:.2f}', 
                 (df_htf['date'].iloc[i], df_htf[f'EMA_{HTF_MA}'].iloc[i]), 
                 textcoords="offset points", 
                 xytext=(0,10), 
                 ha='center', 
                 fontsize=8, 
                 color='black')

plt.xlabel('Date')
plt.ylabel('Price')
plt.title('Close Price and 100-period EMA with Angle Annotations')
plt.legend()
plt.show()

### Calculate RSI

In [ ]:
def gain(value):
    if value < 0:
        return 0
    else:
        return value

In [ ]:
def loss(value):
    if value > 0:
        return 0
    else:
        return abs(value)

In [ ]:

#Calculate price delta
df_ltf['delta'] = df_ltf['close'].diff()

#Classify delta into gain & loss
df_ltf['gain'] = df_ltf['delta'].apply(lambda x:gain(x))
df_ltf['loss'] = df_ltf['delta'].apply(lambda x:loss(x))

#Calculate ema 
df_ltf['ema_gain'] = df_ltf['gain'].ewm(RSI_PERIOD).mean()
df_ltf['ema_loss'] = df_ltf['loss'].ewm(RSI_PERIOD).mean()

#Calculate RSI
df_ltf['rs'] = df_ltf['ema_gain']/df_ltf['ema_loss']
df_ltf['rsi'] = df_ltf['rs'].apply(lambda x: 100 - (100/(x+1)))

In [ ]:
if 'date' in df_ltf.index.names:
    df_ltf.reset_index(inplace=True)

# Plot RSI for validation
plt.figure(figsize=(12, 5))
plt.xticks(rotation=45)

x_axis = df_ltf['date']

x_axis = df_ltf['date']

plt.plot(x_axis, df_ltf['rsi'])
plt.axhline(RSI_BUY_LEVEL, c= (.5,.5,.5), ls='--')
plt.axhline(RSI_SELL_LEVEL, c= (.5,.5,.5), ls='--')

plt.title('RSI ({RSI_PERIOD})', fontweight="bold")
plt.ylim([0, 100])

### Implementing buy/sell

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Initialize signal column
df_ltf['signal'] = 0

# Merge LTF and HTF DataFrames on datetime
df_combined = pd.merge(df_ltf, df_htf[['date', 'ma_angle']], on='date', how='left')

# Forward fill the ma_angle to propagate the 4-hour trend to 15-minute intervals
df_combined['ma_angle'] = df_combined['ma_angle'].ffill()

rsi_shifted = df_combined['rsi'].shift(-1)

# Adjusted buy and sell conditions based on RSI and ma_angle
buy_condition = (df_combined['rsi'] > RSI_BUY_LEVEL) & (rsi_shifted < RSI_BUY_LEVEL) & (df_combined['ma_angle'] > BUY_TREND_ANGLE)
sell_condition = (df_combined['rsi'] < RSI_SELL_LEVEL) & (rsi_shifted > RSI_SELL_LEVEL) & (df_combined['ma_angle'] < SELL_TREND_ANGLE)

# Identify initial buy and sell signals
df_combined.loc[buy_condition, 'signal'] = 1
df_combined.loc[sell_condition, 'signal'] = -1

GAP = 100
last_signal_index = -GAP 
last_signal_value = 0 

for i in range(len(df_combined)):
    current_signal = df_combined.loc[i, 'signal']
    
    if current_signal != 0:
        if (current_signal == last_signal_value) and (i - last_signal_index < GAP):
            df_combined.loc[i, 'signal'] = 0
        else:
            last_signal_index = i
            last_signal_value = current_signal

# Shift signals to the next trading day (optional step depending on your strategy)
df_combined['signal'] = df_combined['signal'].shift()
df_combined['signal'] = df_combined['signal'].fillna(0)

# Prepare the data for plotting
plt.figure(figsize=(16, 10))

# Plot the close price and moving average on the first subplot
plt.subplot(3, 1, 1)
plt.plot(df_combined['date'], df_combined['close'], label='Close Price', color='blue')

# Plot buy and sell signals on the close price chart
plt.scatter(df_combined['date'][df_combined['signal'] == 1], df_combined['close'][df_combined['signal'] == 1], marker='^', color='g', label='Buy Signal', s=100, edgecolor='black', linewidth=1.5)
plt.scatter(df_combined['date'][df_combined['signal'] == -1], df_combined['close'][df_combined['signal'] == -1], marker='v', color='r', label='Sell Signal', s=100, edgecolor='black', linewidth=1.5)

plt.title('Close Price with Buy and Sell Signals')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

# Plot the RSI on the second subplot
plt.subplot(3, 1, 2)
plt.plot(df_combined['date'], df_combined['rsi'], label=f'RSI-{RSI_PERIOD}', color='purple')

# Plot buy and sell signals on the RSI chart at specific levels
plt.scatter(df_combined['date'][df_combined['signal'] == 1], [RSI_BUY_LEVEL] * len(df_combined['date'][df_combined['signal'] == 1]), marker='^', color='g', label='Buy Signal', s=100, edgecolor='black', linewidth=1.5)
plt.scatter(df_combined['date'][df_combined['signal'] == -1], [RSI_SELL_LEVEL] * len(df_combined['date'][df_combined['signal'] == -1]), marker='v', color='r', label='Sell Signal', s=100, edgecolor='black', linewidth=1.5)

plt.axhline(RSI_BUY_LEVEL, color='grey', linestyle='--')
plt.axhline(RSI_SELL_LEVEL, color='grey', linestyle='--')

plt.title(f'RSI-{RSI_PERIOD} with Buy/Sell Signals')
plt.xlabel('Date')
plt.ylabel('RSI')
plt.ylim([0, 100])
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

# Plot the HTF EMA and MA angle on the third subplot
plt.subplot(3, 1, 3)
plt.plot(df_htf['date'], df_htf['close'], label='HTF Close Price', color='blue')
plt.plot(df_htf['date'], df_htf[f'EMA_{HTF_MA}'], label=f'EMA_{HTF_MA}', color='orange')

for i in range(len(df_htf) - 1):
    angle = df_htf['ma_angle'].iloc[i]
    if angle < SELL_TREND_ANGLE:
        plt.axvspan(df_htf['date'].iloc[i], df_htf['date'].iloc[i + 1], color='lightcoral', alpha=0.3)  # Bearish: Light Coral
    elif angle > BUY_TREND_ANGLE:
        plt.axvspan(df_htf['date'].iloc[i], df_htf['date'].iloc[i + 1], color='lightgreen', alpha=0.3)  # Bullish: Light Green
    else:
        plt.axvspan(df_htf['date'].iloc[i], df_htf['date'].iloc[i + 1], color='darkgrey', alpha=0.3)  # Neutral: Dark Grey

plt.title('HTF Close Price and 100 EMA with MA Angles')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)

plt.tight_layout()
plt.show()


import back_testing as bt
bt.backtest_with_drawdown_and_equity_curve(df_combined,maxTrades,initial_balance)
